# 1. Data Collection

In [12]:
#import necessary Library
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

Load data

In [13]:
# Load the dataset
file_path = '../data/MachineLearningRating_v3.txt'
df = pd.read_csv(file_path, sep='|')

Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.


In [14]:
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


# 2. Data preprocessing

Handling missing value

In [15]:
# Calculate the percentage of missing values for each column
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Combine the missing values and percentages into a single DataFrame
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})

# Filter out columns with missing values only
missing_data = missing_data[missing_data['Missing Values'] > 0]

# Display the missing values and their percentage
print(missing_data.sort_values(by='Percentage', ascending=False))


                         Missing Values  Percentage
NumberOfVehiclesInFleet         1000098  100.000000
CrossBorder                      999400   99.930207
CustomValueEstimate              779642   77.956560
Converted                        641901   64.183810
Rebuilt                          641901   64.183810
WrittenOff                       641901   64.183810
NewVehicle                       153295   15.327998
Bank                             145961   14.594670
AccountType                       40232    4.022806
Gender                             9536    0.953507
MaritalStatus                      8259    0.825819
Cylinders                           552    0.055195
cubiccapacity                       552    0.055195
kilowatts                           552    0.055195
NumberOfDoors                       552    0.055195
VehicleIntroDate                    552    0.055195
Model                               552    0.055195
make                                552    0.055195
VehicleType 

In [16]:
# drop the columens that have more than 50% missing values
cols_to_drop = ['NumberOfVehiclesInFleet', 'CrossBorder', 'CustomValueEstimate', 
                'Converted', 'Rebuilt', 'WrittenOff']
df = df.drop(columns=cols_to_drop)


# Impute missing values for categorical columns with the mode
categorical_cols = ['NewVehicle', 'AccountType', 'Bank', 'VehicleType', 
                    'make', 'Model', 'mmcode', 'bodytype', 'Gender', 'MaritalStatus']

for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Impute missing values for numerical columns with the mean
numerical_cols = ['cubiccapacity', 'kilowatts', 'Cylinders', 'NumberOfDoors']
for col in numerical_cols:
    df[col].fillna(df[col].mean(), inplace=True)

# Drop rows with missing values in the 'CapitalOutstanding' column
df.dropna(subset=['CapitalOutstanding'], inplace=True)

# Handle missing values for 'VehicleIntroDate'
default_date = '2014-01-01'
df['VehicleIntroDate'].fillna(default_date, inplace=True)
df['VehicleIntroDate'] = pd.to_datetime(df['VehicleIntroDate'])

# Verify if there are any remaining missing values
print(df.isnull().sum())


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

UnderwrittenCoverID         0
PolicyID                    0
TransactionMonth            0
IsVATRegistered             0
Citizenship                 0
LegalType                   0
Title                       0
Language                    0
Bank                        0
AccountType                 0
MaritalStatus               0
Gender                      0
Country                     0
Province                    0
PostalCode                  0
MainCrestaZone              0
SubCrestaZone               0
ItemType                    0
mmcode                      0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity               0
kilowatts                   0
bodytype                    0
NumberOfDoors               0
VehicleIntroDate            0
AlarmImmobiliser            0
TrackingDevice              0
CapitalOutstanding          0
NewVehicle                  0
SumInsured

Feature Engineering
Create new features that might be relevant for prediction.

In [17]:
# Calculate the percentage of missing values for each column
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Combine the missing values and percentages into a single DataFrame
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})

# Filter out columns with missing values only
missing_data = missing_data[missing_data['Missing Values'] > 0]

# Display the missing values and their percentage
print(missing_data.sort_values(by='Percentage', ascending=False))


Empty DataFrame
Columns: [Missing Values, Percentage]
Index: []


In [18]:
# Example feature engineering
df['PremiumPerClaim'] = df['TotalPremium'] / (df['TotalClaims'] + 1)  # Avoid division by zero
df['ClaimRatio'] = df['TotalClaims'] / (df['TotalPremium'] + 1)  # Avoid division by zero
